In [1]:
from datasets import ContainsTokenSetOOD
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
token_set = set(range(10)) # Edge case/off by one error?
print(len(token_set))
data = ContainsTokenSetOOD(token_set=token_set, seq_len=36, train_size=3600, val_size=360, test_size=360, num_points=None)

10


In [3]:
data.train_data[:5]

(tensor([[ 9,  8, 19,  6, 10,  8,  2, 17,  5, 15, 12,  0,  5, 12,  4, 16,  6,  6,
           1, 10,  7,  2,  1,  5,  8, 14,  8,  7,  3, 10, 13, 19,  3, 17, 12,  3],
         [13, 10, 12,  3, 12, 11, 16,  3,  3, 19, 16, 14, 19, 10, 19, 18,  3, 12,
          13, 12, 10, 16, 19, 19, 12, 16, 14, 13, 15, 15, 15, 18, 14, 18, 18,  3],
         [17, 10,  9,  9,  8,  8,  1,  5,  0, 14,  8, 15,  5, 10,  7,  4,  3,  1,
          17, 16,  1,  3, 10,  9,  4, 18,  8,  8,  3, 17, 13, 11, 18,  5,  2,  6],
         [ 8, 18,  1, 18,  4, 14, 12, 11, 10, 15,  3, 17,  3, 15,  9,  6,  1,  8,
          11, 19, 15, 17,  4,  1, 13, 18, 10,  6, 11, 18, 11, 16, 11,  1,  3, 14],
         [ 3, 17, 16,  9, 14,  9,  3, 14,  3, 11, 13,  9, 14,  6, 16, 15,  7,  7,
           0,  7,  1,  3,  9, 11, 16, 19, 14, 11, 14, 13, 15,  1, 17,  1,  8, 17]]),
 tensor([1, 0, 1, 0, 0]))

In [4]:
[len(set(x.numpy()).intersection(token_set)) for x in data.train_data.dataset.tensors[0][:10]]

[2, 10, 2, 8, 7, 10, 0, 4, 10, 10]

In [5]:
# 0th index is data, 1xt index is label
(
    data.train_data.dataset.tensors[0][:10], 
    [set(x.numpy()).intersection(token_set) for x in data.train_data.dataset.tensors[0][:10]],    
    data.train_data.dataset.tensors[1][:10]
)
pd.DataFrame([len(set(x.numpy()).intersection(token_set)) for x in data.train_data.dataset.tensors[0]]).value_counts()

10    1800
0      200
1      200
2      200
3      200
4      200
5      200
6      200
7      200
8      200
dtype: int64

In [6]:
# 0th index is data, 1xt index is label
(
    data.val_data.dataset.tensors[0][:10], 
    [set(x.numpy()).intersection(token_set) for x in data.val_data.dataset.tensors[0][:10]],    
    data.val_data.dataset.tensors[1][:10]
)
import pandas as pd
pd.DataFrame([len(set(x.numpy()).intersection(token_set)) for x in data.val_data.dataset.tensors[0]]).value_counts()

10    180
0      18
1      18
2      18
3      18
4      18
5      18
6      18
7      18
8      18
9      18
dtype: int64

In [4]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 0    353
1    347
dtype: int64
Val data:
 1    53
0    47
dtype: int64
Test data:
 0    100
1    100
dtype: int64


In [5]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
from transformers import BertConfig, BfBertForSequenceClassification
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-toy.json")
m = BfBertForSequenceClassification(config=config)


env: XLA_PYTHON_CLIENT_PREALLOCATE=false


2023-02-20 16:32:42.493921: W external/org_tensorflow/tensorflow/compiler/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:0: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 4103208960
/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:180: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


In [15]:
np.log(2)

0.6931471805599453

In [23]:
import brunoflow as bf
from jax import numpy as jnp 
x, y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=128)))
# x
outputs = m(bf.Node(jnp.array(x.numpy())), labels=bf.Node(jnp.array(y.numpy())))
outputs.loss, outputs.logits

(node(name: /, val: 0.6931508779525757, grad: 0.0),
 node(name: +, val: [[-8.25370103e-03  1.79829379e-03]
  [ 5.14002983e-03  5.46421972e-04]
  [-3.57060367e-03  1.33854381e-04]
  [-4.50863503e-03  6.43520709e-03]
  [ 3.85447429e-03  1.36725814e-03]
  [ 6.20723097e-03 -2.05880008e-03]
  [ 4.23734868e-03 -6.27395581e-04]
  [-2.27768804e-04  1.45688117e-03]
  [ 4.22999961e-03 -6.27707166e-04]
  [-3.62819363e-03  1.02882383e-04]
  [-1.92662730e-04  1.49287737e-03]
  [-3.62146157e-03  1.02386446e-04]
  [ 6.20649848e-03 -2.05926457e-03]
  [-1.42094679e-03  4.59629064e-03]
  [-3.55485966e-03  1.71255181e-03]
  [ 6.03227597e-03 -2.88169109e-03]
  [ 6.08286588e-03 -2.84436368e-03]
  [ 6.23819605e-03 -2.00822041e-03]
  [ 1.03521300e-03 -2.30820430e-03]
  [ 2.10922607e-03  9.40656289e-03]
  [ 4.23035724e-03 -5.36468870e-04]
  [-2.81263608e-04 -2.57467106e-03]
  [ 4.96772956e-03 -7.86872301e-03]
  [ 2.16012611e-03  9.42399912e-03]
  [-1.41074415e-03  4.52542864e-03]
  [-2.64728442e-04 -2.5835284